In [31]:
import polars as pl 


data = pl.read_csv("output/output.csv", infer_schema_length = 10000)

upstream = data["upstream"]

def get_kmers(seq):

    # Apply function to create full seq in order to compute kmers